In [1]:
from caffe2.python import workspace, model_helper
import numpy as np
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.84747688 0.8486018 ]
  [0.81984266 0.31184698]
  [0.22540864 0.98031111]]

 [[0.25274899 0.89135674]
  [0.67544037 0.06289514]
  [0.26957381 0.67615826]]

 [[0.73467368 0.13392898]
  [0.28905923 0.32732595]
  [0.42949235 0.4001876 ]]

 [[0.77228869 0.73217698]
  [0.67320558 0.64453673]
  [0.54664149 0.68682756]]]
(4, 3, 2)
[[[0.84747688 0.8486018 ]
  [0.81984266 0.31184698]
  [0.22540864 0.98031111]]

 [[0.25274899 0.89135674]
  [0.67544037 0.06289514]
  [0.26957381 0.67615826]]

 [[0.73467368 0.13392898]
  [0.28905923 0.32732595]
  [0.42949235 0.4001876 ]]

 [[0.77228869 0.73217698]
  [0.67320558 0.64453673]
  [0.54664149 0.68682756]]]


In [2]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9].
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

True

In [3]:
print(data)
print(label)

[[0.40822038 0.33604944 0.8884709  ... 0.7320579  0.9553136  0.74147606]
 [0.35785785 0.6863163  0.08476458 ... 0.37981117 0.3280057  0.15287311]
 [0.81047225 0.98456913 0.99941456 ... 0.8349915  0.93825346 0.6201103 ]
 ...
 [0.4962244  0.75960124 0.6028746  ... 0.72015774 0.24056654 0.15676954]
 [0.19160406 0.8653388  0.4089891  ... 0.8297892  0.49838212 0.21671048]
 [0.09471129 0.82387966 0.8505802  ... 0.8766     0.21314591 0.155918  ]]
[2 2 5 6 0 4 8 0 7 8 4 5 1 0 6 6]


In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")


In [5]:
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

In [6]:
fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])


In [6]:
print(m.net.Proto())


name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



In [7]:
print(m.param_init_net.Proto())


name: "my first net_init"
op {
  output: "fc_w"
  name: ""
  type: "XavierFill"
  arg {
    name: "shape"
    ints: 10
    ints: 100
  }
}
op {
  output: "fc_b"
  name: ""
  type: "ConstantFill"
  arg {
    name: "shape"
    ints: 10
  }
}



In [8]:
workspace.RunNetOnce(m.param_init_net)


True

In [9]:
workspace.CreateNet(m.net)


True

In [10]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times

In [11]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))


[[0.09906642 0.11641821 0.09847591 0.09936987 0.1200726  0.08942176
  0.09411614 0.09059852 0.08976386 0.10269673]
 [0.08667116 0.12015975 0.09889498 0.09799227 0.12645313 0.08653833
  0.10251541 0.09169678 0.0933093  0.09576887]
 [0.10018761 0.11611927 0.086586   0.0939147  0.12527959 0.09030907
  0.11082989 0.08773413 0.09450764 0.09453207]
 [0.0927781  0.11810192 0.09369506 0.09073944 0.1137488  0.08263312
  0.10007589 0.0876351  0.10893493 0.11165763]
 [0.08853658 0.10666709 0.09002154 0.09665764 0.10706157 0.09539367
  0.11427453 0.09252985 0.1016605  0.10719694]
 [0.08701625 0.1183838  0.09644759 0.09740038 0.10980542 0.09371648
  0.10209262 0.10300914 0.09566187 0.09646644]
 [0.09237626 0.10814166 0.09742221 0.1010073  0.12689862 0.0826606
  0.09618062 0.09420136 0.09239686 0.10871447]
 [0.09481184 0.10950677 0.10700133 0.08355938 0.1134512  0.08483928
  0.10195229 0.10340553 0.10254233 0.09893006]
 [0.09254903 0.12369885 0.09454206 0.10343386 0.11304748 0.08173081
  0.10722184 

In [12]:
m.AddGradientOperators([loss])


{BlobReference("loss"): BlobReference("loss_autogen_grad"),
 BlobReference("pred"): BlobReference("pred_grad"),
 BlobReference("fc1"): BlobReference("fc1_grad"),
 BlobReference("data"): BlobReference("data_grad"),
 BlobReference("fc_w"): BlobReference("fc_w_grad"),
 BlobReference("fc_b"): BlobReference("fc_b_grad")}

In [13]:
print(m.net.Proto())


name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label